<a href="https://colab.research.google.com/github/AAhmadS/convex_opt/blob/main/99104027_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem 1

#Subproblem 1:

First let's rewrite the likelihood function:<br/>
$[N_t = k]= e^{-λ_t} λ_t ^ k / k!$<br/>
thus the log-likelihood would be as follows:<br/>
$−λ_t + N_tlog λ_t − log N_t!$<br/>
now with the independency assumption, we can find the solution to the problem by minimizing the negative sum of log-likelihood of different t parameters:<br/>

minimize $Σ_{t=1}{24}(λ_t - N_tlog λ_t + log N_t)$<br/>
s.t $λ ≥ 0$<br/>

with variables λ1, . . . , λ24. Note that when Nt > 0, the log λt term imposes an
implicit constraint that λt > 0; we need the explicit inequality λ ≥ 0 to cover
the case when Nt = 0.
This problem is separable in t, so we can separately minimize over each λt
. Let’s
first assume that Nt > 0. Then the minimizer satisfies
 $1 − N_t/λ_t = 0$
 , so
 $λ_t = N_t$
.
For
$Nt = 0$
, we minimize λₜ
, subject to
$λ_t ≥ 0$
. This gives
$λt = 0$.
 So in all cases
the MLE estimate is
$λ_t = N_t$.

#Subproblem 2



In [1]:
import pprint
import scipy
import scipy.linalg
import random
import numpy as np